In [1]:
using GuSTO
using AstrobeeRobot

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/27672ddce0d87eb7829b55d0b54f54d183d7d738-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/27672ddce0d87eb7829b55d0b54f54d183d7d738-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

INFO: Loading HttpServer methods...


In [2]:
robot = Freeflyer()
model = FreeflyerSE2()

FreeflyerSE2(6, 3, 0.05, Any[], Any[], Any[])

In [9]:
env = Table(:stanford);
centers = Vector()
push!(centers, [0.460; 0.315; 0.0])
push!(centers, [0.201; 1.085; 0.0])
push!(centers, [0.540; 2.020; 0.0])
push!(centers, [1.374; 0.196; 0.0])
push!(centers, [1.063; 1.354; 0.0])
push!(centers, [1.265; 2.322; 0.0])
push!(centers, [2.221; 0.548; 0.0])
push!(centers, [2.077; 1.443; 0.0])
push!(centers, [3.098; 1.186; 0.0])
push!(centers, [2.837; 2.064; 0.0])

widths = [0.27; 0.27; 0.127]
inflation = 0.05*ones(3)

for idx in 1:length(centers)
    push!(env.obstacle_set, HyperRectangle(Vec3f0(centers[idx]-0.5*widths-inflation+[0.;0.;0.5*widths[1]]), Vec3f0(widths+2*inflation)))
end

N = 70
x_init = [0.2; 2.4; 0; 0; 0; 0]
x_goal = [3.; 0.5; 0; 0.05; -0.05; 0]

tf_guess = 130.

PD = ProblemDefinition(robot, model, env, x_init, x_goal)

TOP = TrajectoryOptimizationProblem(PD, N, tf_guess, fixed_final_time=true)
TOSgusto = TrajectoryOptimizationSolution(TOP)

SCPP = solve_SCP!(TOSgusto, TOP, solve_gusto_cvx!, init_traj_straightline, "Gurobi", OutputFlag=0);

Academic license - for non-commercial use only


In [10]:
@show TOSgusto.SCPS.successful
@show TOSgusto.SCPS.converged
@show TOSgusto.SCPS.iterations
@show TOSgusto.SCPS.total_time
@show TOSgusto.SCPS.accept_solution
@show TOSgusto.SCPS.prob_status
@show TOSgusto.SCPS.convergence_measure
@show TOSgusto.SCPS.param.alg.omega_vec
@show TOSgusto.SCPS.param.alg.Delta_vec
;

TOSgusto.SCPS.successful = false
TOSgusto.SCPS.converged = true
TOSgusto.SCPS.iterations = 6
TOSgusto.SCPS.total_time = 2.929829477
TOSgusto.SCPS.accept_solution = Bool[true, true, true, true, true, true, true]
TOSgusto.SCPS.prob_status = Symbol[:NA, :Optimal, :Optimal, :Optimal, :Optimal, :Optimal, :Optimal]
TOSgusto.SCPS.convergence_measure = [0.0, 0.103903, 0.0568234, 0.051963, 0.0255017, 0.0107417, 0.00530788]
TOSgusto.SCPS.param.alg.omega_vec = [1.0, 5.0, 25.0, 125.0, 625.0, 3125.0, 15625.0]
TOSgusto.SCPS.param.alg.Delta_vec = [3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]


In [8]:
#### Code for animating free-flyer trajectories
vis = Visualizer()
delete!(vis)

vis[:goal]
setobject!(vis[:goal][:goal], 
    Object(HyperSphere(Point3(x_goal[1:2]..., robot.r), 0.1),
        MeshBasicMaterial(color=RGBA(0,1.0,0.,0.3))))

vis[:table]
table_dim = env.worldAABBmax-env.worldAABBmin
setobject!(vis[:table][:table], 
        Object(HyperRectangle(env.worldAABBmin..., table_dim...),
            MeshBasicMaterial(color=RGBA(0,1.0,0.,0.3))))

vis[:obstacles]
for (idx,obs) in enumerate(env.obstacle_set)
    setobject!(vis[:obstacles][Symbol(string("obs",idx))], 
        Object(obs,MeshBasicMaterial(color=RGBA(1.0,0.,0.,0.3))))
end


vis[:robot]
setobject!(vis[:robot][:base],
    Object(Cylinder(Point3f0(0,0,0.),Point3f0(0.,0.,0.5),Float32(robot.r)),MeshBasicMaterial(color=RGBA(0,0.,1.,0.7))))

up = [0;0;1.]
q0 = vec2quat(up,x_init[3])
Qs = Vector{Vector{Float64}}(0)

speed_factor = 5

for k in 1:speed_factor:N
    q = [quat_inv(quat_multiply(mrp2quat([0.; 0.; tan(TOSgusto.SCPS.traj.X[3,k]/4)]), q0)); [TOSgusto.SCPS.traj.X[1:2,k]; robot.r]]
    push!(Qs,q)
end

anim = Animation()
for k in 1:N
    atframe(anim, vis, 24*k) do frame
        settransform!(frame[:robot], Translation(TOSgusto.traj.X[1,k], 
            TOSgusto.traj.X[2,k], TOSgusto.traj.X[3,k]))
    end
end
setanimation!(vis, anim)
    
plot_in_cell = true
plot_in_cell ? IJuliaCell(vis) : open(vis)

WebIO.Scope("scope-9b5f91c4-535a-4794-bf83-b910a25823c3", WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :iframe), Any[], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict{String,Any}(Pair{String,Any}("height", "100%"),Pair{String,Any}("scrolling", "no"),Pair{String,Any}("src", "javascript:void(0)"),Pair{String,Any}("frameborder", 0))),Pair{Symbol,Any}(:id, "ifr"),Pair{Symbol,Any}(:style, Dict("width"=>"100%","flexGrow"=>"1"))), 0)], Dict{Symbol,Any}(Pair{Symbol,Any}(:style, Dict("flexDirection"=>"column","height"=>"100%","minHeight"=>"400px","display"=>"flex","width"=>"100%","overflow"=>"hidden"))), 1), Dict{String,Tuple{Observables.Observable,Union{Bool, Void}}}(), Set{String}(), nothing, Any[], Dict{Any,Any}(Pair{Any,Any}("_promises", Dict{Any,Any}(Pair{Any,Any}("importsLoaded", Any[WebIO.JSString("function () {\n    var frame = this.dom.querySelector(\"#ifr\");\n    var doc = frame.contentDocument\n    var win = frame.contentWindow\n\n    // Determine if we're running on a Jupyter hosting service\n    // that requires a base URL when retrieving assets\n    var curMatch =\n        window.location.href\n        .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n    curMatch = curMatch ||\n        window.location.href\n        .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n    if (curMatch) {\n        var base = doc.createElement(\"base\");\n        base.setAttribute(\"href\", curMatch[1] + '/');\n        doc.head.appendChild(base);\n    }\n\n    var webio = doc.createElement(\"script\")\n    webio.src = \"pkg/WebIO/webio/dist/bundle.js\"\n    var parent = window\n\n    function resizeIframe() {\n        doc.body.style.padding = '0'\n        doc.body.style.margin = '0'\n        doc.documentElement.height = '100%'\n        doc.body.height = '100%'\n    }\n\n    webio.onload = function () {\n        win.WebIO.sendCallback = parent.WebIO.sendCallback; // Share stuff\n        win.WebIO.scopes = parent.WebIO.scopes\n        win.WebIO.obsscopes = parent.WebIO.obsscopes\n        win.WebIO._connected = true\n        doc.body.innerHTML = \"<html><body>\" + \"<div class='display:none'></div><unsafe-script style='display:none'>\\nWebIO.mount(this.previousSibling,{&quot;props&quot;:{},&quot;nodeType&quot;:&quot;Scope&quot;,&quot;type&quot;:&quot;node&quot;,&quot;instanceArgs&quot;:{&quot;imports&quot;:{&quot;data&quot;:[{&quot;name&quot;:null,&quot;type&quot;:&quot;js&quot;,&quot;url&quot;:&quot;/assetserver/4b08ff53bcd7c767510e4fdfe439fd147a82ded2-main.min.js&quot;}],&quot;type&quot;:&quot;async_block&quot;},&quot;id&quot;:&quot;scope-646983f4-5b9c-4996-bc9d-ea0e53f56b3d&quot;,&quot;handlers&quot;:{&quot;meshcat-command&quot;:[(function (val){return this.viewer.handle_command_message({&quot;data&quot;:val})})],&quot;_promises&quot;:{&quot;importsLoaded&quot;:[(function (mc){var element=this.dom.children[0]; this.viewer=new mc.Viewer(element); WebIO.setval({&quot;name&quot;:&quot;meshcat-request&quot;,&quot;scope&quot;:&quot;scope-646983f4-5b9c-4996-bc9d-ea0e53f56b3d&quot;,&quot;id&quot;:&quot;ob_05&quot;,&quot;type&quot;:&quot;observable&quot;},String(Date.now())); return window.document.body.style.margin=&quot;0&quot;})]}},&quot;systemjs_options&quot;:null,&quot;observables&quot;:{&quot;meshcat-command&quot;:{&quot;sync&quot;:false,&quot;id&quot;:&quot;ob_04&quot;,&quot;value&quot;:[131,167,111,112,116,105,111,110,115,130,164,112,108,97,121,195,171,114,101,112,101,116,105,116,105,111,110,115,1,170,97,110,105,109,97,116,105,111,110,115,145,130,164,99,108,105,112,131,164,110,97,109,101,167,100,101,102,97,117,108,116,163,102,112,115,30,166,116,114,97,99,107,115,146,131,164,110,97,109,101,169,46,112,111,115,105,116,105,111,110,164,107,101,121,115,220,0,70,130,164,116,105,109,101,24,165,118,97,108,117,101,147,203,63,201,153,153,153,153,153,154,203,64,3,51,51,51,51,51,51,203,0,0,0,0,0,0,0,0,130,164,116,105,109,101,48,165,118,97,108,117,101,147,203,63,202,219,231,50,124,165,40,203,64,3,57,189,229,3,68,80,203,62,193,